In [42]:
from io import BytesIO
from IPython.display import clear_output, Image, display
from PIL import Image as Img
import cv2
from os import listdir
from os.path import isfile, join
import numpy as np

def padronizar_imagem(imagem):
    imagem = cv2.cvtColor(imagem, cv2.COLOR_BGR2RGB)
    imagem = cv2.resize(imagem, (400,300))
    return imagem

def exibir_video(frame):
    img = Img.fromarray(frame, "RGB")
    buffer = BytesIO()
    img.save(buffer, format="JPEG")
    display(Image(data=buffer.getvalue()))
    clear_output(wait=True)

Vamos agora criar um loop infinito (ou até que seja acionada a interrupção do Kernel, no Notebook clique no menu Kernal e depois em “Interrupt”), para exibir a câmera do notebook.

Repare que o valor no cv2.VideoCapture(0) refere-se à primeira câmera conectada no seu computador. Se houver mais câmeras você pode trocar este número.

Se durante a execução houver algum travamento da câmera, crie uma célula nova no Notebook e envie o comando captura_video.release().}

In [41]:
captura_video = cv2.VideoCapture(0)

pathfile_haar = "/home/svtrp/anaconda3/envs/wenisSVTRP/share/opencv4/haarcascades/haarcascade_frontalface_default.xml"
classificador_face = cv2.CascadeClassifier(pathfile_haar)

contador = 0

try:
    while(True):
        captura_ok, frame = captura_video.read()

        if captura_ok:
            frame = padronizar_imagem(frame)
            frame_gray = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)

            faces = classificador_face.detectMultiScale(frame_gray, 1.3, 5)

            if len(faces) > 0:
                contador += 1
                if contador <= 100:
                    for (x,y,w,h) in faces:
                        roi = frame_gray[y:y+h, x:x+w]
                        cv2.resize(roi, (200, 200), interpolation=cv2.INTER_LANCZOS4)
                        cv2.imwrite("imagens/wenis/" + str(contador) + ".png", roi)
                        cv2.putText(frame, "Coletado " + str(contador) + " faces", (20,20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,0), 2)
                else:
                    cv2.putText(frame, "ConcluIdo", (20,20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,0), 2)

            exibir_video(frame)

except KeyboardInterrupt:
    captura_video.release()
    print("Interrompido")

Interrompido


OK, até aqui você deve estar se vendo na tela do Notebook.

Agora, vamos aplicar um classificador de cascata de Haar de face frontal e extrair 100 exemplos. Vamos aproveitar para salvar a imagem já no formato de escala de cinza.

Veja que adicionamos no código anterior as etapas de extração de região de interesse, redimensionamento para 200x200 e exportação de arquivo externo.

Uma vez que atingir a quantidade limite, avisamos na tela para que seja interrompido. Do contrário, exibimos o contador até alcançar 100.

Antes de prosseguir, crie uma pasta com seu nome dentro da pasta imagens. No meu caso usei meu nome.

OK, até aqui você deve estar se vendo na tela do Notebook.

Agora, vamos aplicar um classificador de cascata de Haar de face frontal e extrair 100 exemplos. Vamos aproveitar para salvar a imagem já no formato de escala de cinza.

Veja que adicionamos no código anterior as etapas de extração de região de interesse, redimensionamento para 200x200 e exportação de arquivo externo.

Uma vez que atingir a quantidade limite, avisamos na tela para que seja interrompido. Do contrário, exibimos o contador até alcançar 100.

Antes de prosseguir, crie uma pasta com seu nome dentro da pasta imagens. No meu caso usei meu nome.


Agora que temos as imagens, vamos construir nosso classificador com o algoritmo de LBPH.

In [43]:
faces_caminho_treinamento = "imagens/wenis/"
lista_faces_treinamento = [f for f in listdir(faces_caminho_treinamento) if isfile(join(faces_caminho_treinamento, f))]

In [44]:
dados_treinamento, sujeitos = [], []

for i, arquivo in enumerate(lista_faces_treinamento):
    imagem_caminho = faces_caminho_treinamento + arquivo
    imagem = cv2.imread(imagem_caminho, cv2.IMREAD_GRAYSCALE)
    dados_treinamento.append(imagem)
    sujeitos.append(0)

sujeitos = np.asarray(sujeitos, dtype=np.int32)


Só existe uma pessoa, por isso vamos cravar o número 0

In [45]:
model_lbph = cv2.face.LBPHFaceRecognizer_create()
model_lbph.train(dados_treinamento, sujeitos)

Finalmente, vamos copiar o código que usamos para extrair os exemplos e adaptar o modelo para inferir, frame a frame, a similaridade do rosto apresentado com o que treinamos.

Geralmente, a similaridade não deve passar de 40. Faça seu teste.

Agora, utilizando seu smartphone, procure por outro rosto e coloque na frente da câmera do computador. Perceba que a similaridade aumentou bastante, pois é um rosto bem diferente do que treinamos.

In [46]:
captura_video = cv2.VideoCapture(0)

pathfile_haar = "/home/svtrp/anaconda3/envs/wenisSVTRP/share/opencv4/haarcascades/haarcascade_frontalface_default.xml"
classificador_face = cv2.CascadeClassifier(pathfile_haar)
contador = 0

try:
    while(True):
        captura_ok, frame = captura_video.read()

        if captura_ok:
            frame = padronizar_imagem(frame)
            frame_gray = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
            faces = classificador_face.detectMultiScale(frame_gray, 1.3, 5)

            if len(faces) > 0:
                for (x,y,w,h) in faces:
                    roi = frame_gray[y:y+h, x:x+w]
                    roi = cv2.resize(roi, (200, 200), interpolation=cv2.INTER_LANCZOS4)
                    predicao = model_lbph.predict(roi)
                    cv2.putText(frame, "Similaridade " + str(predicao[1]), (x,y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,0), 2)

            exibir_video(frame)

except KeyboardInterrupt:
    captura_video.release()
    print("Interrompido")

Interrompido
